<a href="https://colab.research.google.com/github/msakarvadia/Machine-Learning/blob/master/LC_LoanDefaultRatePrediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Command to run local server:

jupyter notebook --no-browser --allow-root --NotebookApp.allow_origin='https://colab.research.google.com' --NotebookApp.token=''  --NotebookApp.disable_check_xsrf=True


the above is the comand line to connect to local host when starting a jupyter notebook (remember to be in the corrent directory)

#UPLOAD DATA:

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import io
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.utils import np_utils
from sklearn.preprocessing import OneHotEncoder

Using TensorFlow backend.


In [2]:
cd /mnt/c/Users/mansi/Desktop/High\ School/senior/Applied\ AI/Loan\ data/lending\ club\ data

/mnt/c/Users/mansi/Desktop/High School/senior/Applied AI/Loan data/lending club data


In [3]:
ls

desktop.ini*                                 LoanStats_LendingClub_2016Q4.csv*
LCDataDictionary.xlsx*                       LoanStats_LendingClub_2017Q1.csv*
LoanStats3a_securev1_LendingClub_07-11.csv*  LoanStats_LendingClub_2017Q2.csv*
LoanStats3b_securev1_LendingClub_12-13.csv*  LoanStats_LendingClub_2017Q3.csv*
LoanStats3c_LendingClub_2014.csv*            LoanStats_LendingClub_2017Q4.csv*
LoanStats3d_LendingClub_2015.csv*            LoanStats_LendingClub_2018Q1.csv*
LoanStats_LendingClub_2016Q1.csv*            LoanStats_LendingClub_2018Q2.csv*
LoanStats_LendingClub_2016Q2.csv*            LoanStats_LendingClub_2018Q3.csv*
LoanStats_LendingClub_2016Q3.csv*            RejectStatsA.csv*


In [4]:
#for host run time
'''from google.colab import drive
drive.mount('/content/drive')'''

"from google.colab import drive\ndrive.mount('/content/drive')"

In [5]:
#for host run time
'''#!pwd
%cd drive/My\ Drive
%cd Colab\ Notebooks
%cd Loan\ Project
# we are in the "Loan Project directory" - if there is an issue use "%cd ... to reenter directory"
#!ls
#!cat LoanStats3a_securev1_LendingClub_07-11.csv'''

'#!pwd\n%cd drive/My\\ Drive\n%cd Colab\\ Notebooks\n%cd Loan\\ Project\n# we are in the "Loan Project directory" - if there is an issue use "%cd ... to reenter directory"\n#!ls\n#!cat LoanStats3a_securev1_LendingClub_07-11.csv'

In [6]:
#data is from Lending Club - I have renamed csv files
df_07_11 = pd.read_csv('LoanStats3a_securev1_LendingClub_07-11.csv', sep=",", skiprows=1)
df_12_13 = pd.read_csv('LoanStats3b_securev1_LendingClub_12-13.csv', sep=",", skiprows=1)
df_14 = pd.read_csv("LoanStats3c_LendingClub_2014.csv", sep=",", skiprows=1)
df_15 = pd.read_csv('LoanStats3d_LendingClub_2015.csv', sep=",", skiprows=1)
df_16Q1 = pd.read_csv('LoanStats_LendingClub_2016Q1.csv', sep=",", skiprows=1)
df_16Q2 = pd.read_csv('LoanStats_LendingClub_2016Q2.csv', sep=",", skiprows=1)
df_16Q3 = pd.read_csv('LoanStats_LendingClub_2016Q3.csv', sep=",", skiprows=1)
df_16Q4 = pd.read_csv('LoanStats_LendingClub_2016Q4.csv', sep=",", skiprows=1)
df_17Q1 = pd.read_csv('LoanStats_LendingClub_2017Q1.csv', sep=",", skiprows=1)
df_17Q2 = pd.read_csv('LoanStats_LendingClub_2017Q2.csv', sep=",", skiprows=1)
df_17Q3 = pd.read_csv('LoanStats_LendingClub_2017Q3.csv', sep=",", skiprows=1)
df_17Q4 = pd.read_csv('LoanStats_LendingClub_2017Q4.csv', sep=",", skiprows=1)
df_18Q1 = pd.read_csv('LoanStats_LendingClub_2018Q1.csv', sep=",", skiprows=1)
df_18Q2 = pd.read_csv('LoanStats_LendingClub_2018Q2.csv', sep=",", skiprows=1)
df_18Q3 = pd.read_csv('LoanStats_LendingClub_2018Q3.csv', sep=",", skiprows=1)

/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (0,49) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (0,49,129,130,131,134,135,136,139) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (0,19) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (0,19,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, 

#PROCESS THE DATA:

In [7]:
frames = [df_07_11, df_12_13, df_14, df_15, df_16Q1, df_16Q2, df_16Q3, df_16Q4, df_17Q1, df_17Q2, df_17Q3, df_17Q4, df_18Q1,df_18Q2,df_18Q3]

result = pd.concat(frames, ignore_index=True)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:3: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  This is separate from the ipykernel package so we can avoid doing imports until


In [8]:
#shuffle enties - if you use dates at some point, dont shuffle
result = result.reindex(np.random.permutation(result.index)) #for now we arn't using data, so we shuffle
result.shape

(2132289, 151)

In [9]:
result.drop(columns=['id', 'member_id','url','zip_code', 'issue_d'], inplace=True)
result.shape

(2132289, 146)

In [10]:

#"loan_status" is the column heading of whether it defulted or not
x = result.copy()
#labels = df.loc[:,"loan_status"]
y = result['loan_status'].copy()
y.shape[0]

2132289

In [11]:
import multiprocessing as mp
from multiprocessing import Pool


label = []

 
for i in range(y.shape[0]):
  if y[i] not in label:
    label.append(y[i])

for i in range(len(label)):
  print(label[i])
  
print("There are", len(label), "labels")

Fully Paid
Charged Off
nan
Does not meet the credit policy. Status:Fully Paid
Does not meet the credit policy. Status:Charged Off
Late (31-120 days)
Current
Late (16-30 days)
In Grace Period
Default
There are 10 labels


##NOTES about loan_status (aka. labels):


*   if label is "nan" - we throw that loan out of the training set
*   Current means that the loan is still being paid off, and therefore we cannot use for out training - might be useful to comeback and revisit in the future (maybe predict outcome now) - will be thrown out of training set
*   fully paid  - successful loan


---


*   what is "charged off" vs "default"
 *   "Loans that are in "Default" are loans for which borrowers have failed to make payments for an extended period of time. A loan becomes “Charged Off” when there is no longer a reasonable expectation of further payments.  Charge Off typically occurs when a loan is 120 days or more past due and there is no reasonable expectation of sufficient payment to prevent the charge off.  In certain circumstances, loans may be charged off at an earlier or later date.  Please note, loans for which borrowers have filed for bankruptcy may be charged off earlier based on the date of bankruptcy notification.
 A loan that is in “Default” will still appear in your Notes, in the status of “Default,” while a loan that has been “Charged Off” will appear as charged off, and the remaining principal balance of the Note will be deducted from your account balance.  Learn more about what happens when a loan is charged off"
 *   Because in the case of "charged off" vs "default" the lender has extremely low chance of recovering their investment, we will treat both of these classifications as if they are the same thing

---


*   what does it mean if it "does not meet the credit policy"
 *   Lending Club doesn't make clear the meaning of this statement, but from general internet research most ppl just ommit the "does not meet the credit policy" part and use the second half of the classification
---
*   what does "grace period" mean - should we potentially throw those away?
 * Loan is past due but within the 15-day grace period. - we will throw these away because for training data it isn't useful since the final status of loan isn't avaliable
*   if its late - does that mean status is still being finalized? - should we throw away?
 * if it is late we don't use this loan in training set cus finalized status isn't avalible to confirm predictions



In [12]:
result = result[~result['loan_status'].isin(['Late (31-120 days)', 'Current', 'Late (16-30 days)', 'In Grace Period'])]
# above gets rid of rows that have loan status' that we don't want to consider
result = result[~result['loan_status'].isna()] 

#make labels binary
pay_status = {'Fully Paid':1, 'Charged Off':0, 'Does not meet the credit policy. Status:Fully Paid':1, 'Does not meet the credit policy. Status:Charged Off':0, 'Default':0}
result.loan_status = [pay_status[item] for item in result.loan_status]


# above gets rid of rows that don't have a valid loan status
result.shape

(1201050, 146)

In [13]:
#DROPPING instances of larger ("paid") class - didn't show much effect
#result = result.drop(result[result.loan_status.eq(1)].sample(frac=.75).index)
result.shape

(1201050, 146)

In [57]:
#"loan_status" is the column heading of whether it defulted or not
x = result.copy()


#labels = df.loc[:,"loan_status"]
y = result['loan_status'].copy()

#get rid of labels from test set
x.drop(columns=['loan_status'], inplace=True) 

#for now, we are gonna drop all columns that have even one NAN value - this signifigantly reduces training data (so its something we want to comeback and deal with later)
x.dropna(axis='columns', inplace=True)
print(list(x))

['addr_state', 'application_type', 'collection_recovery_fee', 'debt_settlement_flag', 'disbursement_method', 'funded_amnt', 'funded_amnt_inv', 'grade', 'hardship_flag', 'home_ownership', 'initial_list_status', 'installment', 'int_rate', 'last_pymnt_amnt', 'loan_amnt', 'out_prncp', 'out_prncp_inv', 'policy_code', 'purpose', 'pymnt_plan', 'recoveries', 'revol_bal', 'sub_grade', 'term', 'total_pymnt', 'total_pymnt_inv', 'total_rec_int', 'total_rec_late_fee', 'total_rec_prncp', 'verification_status']


In [0]:
x['term'] = x['term'].map(lambda x: x.lstrip('%').rstrip('months'))
x['int_rate'] = x["int_rate"].map(lambda x: x.rstrip("%"))
int_rate = x['int_rate'].str.replace(' ', '')
x.int_rate = x.int_rate.astype(float)
#x['revol_util'] = x['revol_util'].str.replace('%', '') #format changes bc column as floats and ints, also has NaN values, so it was removed above
#print(x["revol_util"])
#x.drop(columns=['int_rate'], inplace=True)

In [0]:
#y = np_utils.to_categorical(y)
#Convert categorical variable into dummy/indicator variables:
x =pd.get_dummies(x, drop_first=True)

In [64]:
print(list(x))
print(x.shape)
y.shape

['collection_recovery_fee', 'funded_amnt', 'funded_amnt_inv', 'installment', 'int_rate', 'last_pymnt_amnt', 'loan_amnt', 'out_prncp', 'out_prncp_inv', 'policy_code', 'recoveries', 'revol_bal', 'total_pymnt', 'total_pymnt_inv', 'total_rec_int', 'total_rec_late_fee', 'total_rec_prncp', 'addr_state_AL', 'addr_state_AR', 'addr_state_AZ', 'addr_state_CA', 'addr_state_CO', 'addr_state_CT', 'addr_state_DC', 'addr_state_DE', 'addr_state_FL', 'addr_state_GA', 'addr_state_HI', 'addr_state_IA', 'addr_state_ID', 'addr_state_IL', 'addr_state_IN', 'addr_state_KS', 'addr_state_KY', 'addr_state_LA', 'addr_state_MA', 'addr_state_MD', 'addr_state_ME', 'addr_state_MI', 'addr_state_MN', 'addr_state_MO', 'addr_state_MS', 'addr_state_MT', 'addr_state_NC', 'addr_state_ND', 'addr_state_NE', 'addr_state_NH', 'addr_state_NJ', 'addr_state_NM', 'addr_state_NV', 'addr_state_NY', 'addr_state_OH', 'addr_state_OK', 'addr_state_OR', 'addr_state_PA', 'addr_state_RI', 'addr_state_SC', 'addr_state_SD', 'addr_state_TN', '

(1201050,)

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)

seed = 45
np.random.seed(seed)

In [66]:
print(y_train.size + y_test.size)
X_train

1201050


,collection_recovery_fee,funded_amnt,funded_amnt_inv,installment,int_rate,last_pymnt_amnt,loan_amnt,out_prncp,out_prncp_inv,policy_code,...,sub_grade_F4,sub_grade_F5,sub_grade_G1,sub_grade_G2,sub_grade_G3,sub_grade_G4,sub_grade_G5,term_ 60,verification_status_Source Verified,verification_status_Verified
1552351,0.0000,10000.0,10000.00,326.97,10.91,8556.12,10000.0,0.0,0.0,1.0,...,0,0,0,0,0,0,0,0,0,1
292487,99.9846,5000.0,5000.00,181.37,18.24,181.37,5000.0,0.0,0.0,1.0,...,0,0,0,0,0,0,0,0,0,0
512146,0.0000,10500.0,10500.00,325.46,7.26,5277.63,10500.0,0.0,0.0,1.0,...,0,0,0,0,0,0,0,0,1,0
249294,0.0000,15000.0,14900.00,474.70,8.67,948.76,15000.0,0.0,0.0,1.0,...,0,0,0,0,0,0,0,0,1,0
460996,0.0000,28000.0,28000.00,741.37,19.97,4980.38,28000.0,0.0,0.0,1.0,...,0,0,0,0,0,0,0,1,0,1
957821,0.0000,17425.0,17375.00,643.42,19.53,643.42,17425.0,0.0,0.0,1.0,...,0,0,0,0,0,0,0,0,0,1
191846,28.7323,25000.0,25000.00,624.95,17.27,624.95,25000.0,0.0,0.0,1.0,...,0,0,0,0,0,0,0,1,0,1
61209,1385.6634,35000.0,35000.00,1249.59,17.10,1249.59,35000.0,0.0,0.0,1.0,...,0,0,0,0,0,0,0,0,0,1
870265,0.0000,20000.0,20000.00,411.98,8.67,18242.97,20000.0,0.0,0.0,1.0,...,0,0,0,0,0,0,0,1,0,1
1332276,0.0000,10500.0,10500.00,255.29,15.99,9220.06,10500.0,0.0,0.0,1.0,...,0,0,0,0,0,0,0,1,0,0


# SHALLOW NN:

In [0]:

my_model = Sequential()
my_model.add(Dense(150, input_dim=134, activation='sigmoid'))
my_model.add(Dense(150,  activation='sigmoid'))
my_model.add(Dense(150,  activation='sigmoid'))
my_model.add(Dense(150,  activation='sigmoid'))
my_model.add(Dense(1, activation='softmax')) #when i one hot encode the answer key - u need output nueron number to be 2 not 1

my_model.compile(loss='binary_crossentropy', optimizer='sgd', metrics=['accuracy']) #need to one hot encode the answer key cus im a dumb dumb

In [68]:
scores = my_model.evaluate(X_train, y_train, verbose=0)
print("Accuracy on training data: %.2f%%" % (scores[1]*100))

scores = my_model.evaluate(X_test, y_test, verbose=0)
print("Accuracy on test data: %.2f%%" % (scores[1]*100))

y_pred = my_model.predict(X_train)

from sklearn.metrics import confusion_matrix
confusion_matrix(y_train, y_pred)

#model is never predicting that the person will not pay

Accuracy on training data: 79.93%
Accuracy on test data: 79.93%


array([[     0, 161500],
       [     0, 643203]])

#BASIC CNN:

In [69]:
x_conv = np.expand_dims(x, axis=2) #need to reshape for CONV ONLY
print(x_conv.shape)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x_conv, y, test_size=0.33, random_state=42)

seed = 45
np.random.seed(seed)

(1201050, 134, 1)


In [0]:
from keras.models import Sequential
from keras.layers import Dense, Conv1D, Flatten
#create model
CNNmodel = Sequential()
#add model layers
CNNmodel.add(Conv1D(64, kernel_size=2, activation='relu', input_shape=(134,1)))
CNNmodel.add(Conv1D(32, kernel_size=2, activation='relu'))
CNNmodel.add(Flatten())
CNNmodel.add(Dense(1, activation='softmax'))

CNNmodel.compile(loss='binary_crossentropy', optimizer='sgd', metrics=['accuracy'])

In [71]:
scores = CNNmodel.evaluate(X_train, y_train, verbose=0)
print("Accuracy on training data: %.2f%%" % (scores[1]*100))

scores = CNNmodel.evaluate(X_test, y_test, verbose=0)
print("Accuracy on test data: %.2f%%" % (scores[1]*100))

y_predCNN = CNNmodel.predict(X_train)
confusion_matrix(y_train, y_predCNN)

Accuracy on training data: 79.93%
Accuracy on test data: 79.93%


array([[     0, 161500],
       [     0, 643203]])

#Naive Bayes Classifier
##Gaussian Distribution
###Continuous variables


*   We can also test multinomial and bernoulli models from scikit-learn just to see how they perform for kicks and giggles
*   https://www.youtube.com/watch?v=r1in0YNetG8
  *  the above is a video for gaussian naive bayes
  * use pdf (probability density function) (can google the equation)



In [72]:
x_Naive = pd.concat([x, y], axis=1)
x_Naive = x_Naive.drop(x_Naive[x_Naive.loan_status.eq(1)].sample(frac=.80).index)
x_Naive = x_Naive.drop(x_Naive.sample(frac=.90).index)
y_Naive = x_Naive['loan_status'].copy()
x_Naive.drop(columns=['loan_status'], inplace=True) 
x_Naive.shape

(43304, 134)

In [0]:
#need a function to find class_size
def class_sizeFinder(df, Class):
  return(df[Class].size)

#need function to determine mean of each class
#for Class you need to pass in a df indexed into the class in a way that can be indexed into each individual feature
def mean(df, Class):
  return(df[Class].mean())
  

#need function to determine variance of each class
def stand_dev(df, Class):
  return(df[Class].std())

#need probability density function

In [0]:
#split into train/test set
X_train, X_test, y_train, y_test = train_test_split(x_Naive, y_Naive, test_size=0.33, random_state=42)

In [0]:
#split all instances with paid vs defaulted loans

x_one = pd.DataFrame()
x_zero = pd.DataFrame()

k = 0
for i in y_train:
  if i == 1:
    x_one = x_one.append(X_train[k:k+1])
  if i ==0:
    x_zero = x_zero.append(X_train[k:k+1])
  k+=1

In [112]:
#verify that the splitting was accurate
print(x_one.shape)
print(x_zero.shape)
print(x_one.shape[0]+x_zero.shape[0])
y_train.size

(12925, 134)
(16088, 134)
29013


29013

In [0]:
#calculate mean/variance for both classes
mean_one = []
mean_zero = []
std_one = []
std_zero = []

for i in x_zero:
  mean_one.append(mean(x_one, i))
  mean_zero.append(mean(x_zero, i))
  std_one.append(stand_dev(x_one, i))
  std_zero.append(stand_dev(x_one,i))

In [0]:
#probability density function
def pdf(feature, mean, std):
  return(1/sqrt(2*math.pi*std**2)*exp(-1/2*((feature-mean)/std**2)**2))

In [0]:
#calculate predictions for class 1 for test set

#calculate predition for class 2 for test set